In [1]:
import pandas as pd
import numpy as np
from os import listdir

In [2]:
df1 = pd.read_csv("./csv/events_up_to_01062018.csv",low_memory=False)

In [3]:
df2 = pd.read_csv("./csv/labels_training_set.csv")

In [4]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

In [5]:
def modelo(obj):
    if(pd.isnull(obj)):
        return obj
    
    list = obj.split(" ")
    if len(list) == 1:
        return obj
    if "iPhone" in list:
        return "iPhone"
    if "Samsung" in list:
        return "Samsung"
    if "Motorola" in list:
        return "Motorola"
    if "Lenovo" in list:
        return "Lenovo"
    if "Sony" in list:
        return "Sony"
    if "LG" in list:
        return "LG"
    if "Asus" in list:
        return "Asus"
    if "iPad" in list:
        return "iPad"
    if "Quantum" in list:
        return "Quantum"
    return obj

df1['model'] = df1['model'].apply(modelo)

In [6]:
df1['model'].value_counts()

iPhone              593603
Samsung             501307
Motorola            162914
LG                   24814
Sony                 15599
Lenovo               11382
iPad                  5663
Asus                  4536
Quantum               1693
Outros TV LED 15         1
Xiaomi Redmi 2           1
Name: model, dtype: int64

In [7]:
df1['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

# Aalisis para evento convercion

In [8]:
df_a = df1.loc[df1['event']=='conversion',:]

In [9]:
df_a.isnull().sum()

timestamp                      0
event                          0
person                         0
url                         7091
sku                            0
model                          0
condition                      0
storage                        0
color                          0
skus                        6918
search_term                 7091
staticpage                  7091
campaign_source             7091
search_engine               7091
channel                     7091
new_vs_returning            7091
city                        7091
region                      7091
country                     7091
device_type                 7091
screen_resolution           7091
operating_system_version    7091
browser_version             7091
dtype: int64

In [10]:
df_a_model = df_a.loc[:,['person','model','storage','color']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model"})
        
df_a_storage = df_a.loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage"})
        
df_a_condition = df_a.loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition"})
        
df_a_color = df_a.loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color"})

In [11]:
df_datos_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\

In [12]:
#df_datos_convercion.to_csv("df_datos_convercion.csv", index = True, header = True, sep = ',', 
#          encoding = 'utf-8-sig')

# Analisis para los eventos distintos de convercion

In [13]:
df_a = df1.loc[df1['event']!='conversion',:]

In [14]:
df_a_model = df_a.dropna(subset=['model']).loc[:,['person','model']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model_no_convercion"})


df_a_storage = df_a.dropna(subset=['storage']).loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage_no_convercion"})
        
        
df_a_condition = df_a.dropna(subset=['condition']).loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition_no_convercion"})
        
        
df_a_color = df_a.dropna(subset=['color']).loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color_no_convercion"})

In [15]:
df_datos_no_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\

In [16]:
#df_datos_no_convercion.to_csv("df_datos_no_convercion.csv", index = True, header = True, sep = ',', 
#          encoding = 'utf-8-sig')

# Mergeo de los DF y procesamiento

In [18]:
DF_a_procesar = df_datos_convercion.merge(df_datos_no_convercion,how='left',on='person')\
                                    .merge(df2,how='left',on='person')

In [19]:
DF_a_procesar = DF_a_procesar.dropna(subset=['label'])

In [20]:
def procesar_df(df):
    df_2 = df
    for element in df.index.tolist():
        label = df.at[element,'label']
        if label==0:
            person = df.at[element,'person']
            df_2.loc[df_2.person==person,['model']] = df.at[element,'model_no_convercion']
            df_2.loc[df_2.person==person,['storage']] = df.at[element,'storage_no_convercion']
            df_2.loc[df_2.person==person,['condition']] = df.at[element,'condition_no_convercion']
            df_2.loc[df_2.person==person,['color']] = df.at[element,'color_no_convercion']
    df_2 = df_2.loc[:,['person','model','storage','condition','color','label']]
    return df_2

In [21]:
dffinal=procesar_df(DF_a_procesar)

Le agrego el serch_engine a la persona

In [22]:
df_a_search_engine = df_a.dropna(subset=['search_engine']).loc[:,['person','search_engine']].groupby('person')\
                .apply(lambda x: x['search_engine'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"search_engine"})
df_a_search_engine.head()

,person,search_engine
0,000ba417,Google
1,000c79fe,Google
2,000e4d9e,Google
3,000e619d,Google
4,0019e639,Google


In [23]:
dffinal = dffinal.merge(df_a_search_engine,how='left',on='person')

In [25]:
dffinal = dffinal.dropna(subset=['search_engine'])
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1712 entries, 0 to 2163
Data columns (total 7 columns):
person           1712 non-null object
model            1712 non-null object
storage          1712 non-null object
condition        1712 non-null object
color            1712 non-null object
label            1712 non-null float64
search_engine    1712 non-null object
dtypes: float64(1), object(6)
memory usage: 107.0+ KB


Escribo el data frame para usar machine learning

In [26]:
dffinal.to_csv("df_a_procesar.csv", index = True, header = True, sep = ',', 
          encoding = 'utf-8-sig')